<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Data-wrangling" data-toc-modified-id="Data-wrangling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data wrangling</a></span><ul class="toc-item"><li><span><a href="#Null-values-in-the-train-set" data-toc-modified-id="Null-values-in-the-train-set-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Null values in the train set</a></span></li><li><span><a href="#Null-values-in-the-test-set" data-toc-modified-id="Null-values-in-the-test-set-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Null values in the test set</a></span></li></ul></li><li><span><a href="#Text-vectorizer" data-toc-modified-id="Text-vectorizer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Text vectorizer</a></span><ul class="toc-item"><li><span><a href="#Count-Vectorizer" data-toc-modified-id="Count-Vectorizer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Count Vectorizer</a></span></li><li><span><a href="#TfidVectorizer" data-toc-modified-id="TfidVectorizer-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>TfidVectorizer</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
#import dataframe_image as dfi
import os
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import statsmodels.api as sm
import re
import nltk
import spacy
import math

from pandas_profiling.utils.cache import cache_file

from sklearn import metrics, datasets, linear_model, preprocessing
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, classification_report, precision_recall_curve, mean_squared_error, f1_score, plot_confusion_matrix  
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import scale, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from textblob import TextBlob
from wordcloud import WordCloud
import scipy.sparse

#nltk.download('punkt')
#print(os.getcwd())
path="/Users/atousachaharsoughshirazi/drug_review"
os.chdir(path)
file_train="drugsComTrain_raw.csv"
df=pd.read_csv(file_train, header=0)
file_test="drugsComTest_raw.csv"
df_test=pd.read_csv(file_test, header=0)

# Introduction
* Classification: Can you predict the patient's condition based on the review?
* Regression: Can you predict the rating of the drug based on the review?
* Sentiment analysis: What elements of a review make it more helpful to others? Which patients tend to have more negative reviews? Can you determine if a review is positive, neutral, or negative?
* Data visualizations: What kind of drugs are there? What sorts of conditions do these patients have?

# Data wrangling

In [2]:
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [3]:
df.dtypes

uniqueID        int64
drugName       object
condition      object
review         object
rating          int64
date           object
usefulCount     int64
dtype: object

In [4]:
df.columns

Index(['uniqueID', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')

In [5]:
df.shape

(161297, 7)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     161297 non-null  int64 
 1   drugName     161297 non-null  object
 2   condition    160398 non-null  object
 3   review       161297 non-null  object
 4   rating       161297 non-null  int64 
 5   date         161297 non-null  object
 6   usefulCount  161297 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.6+ MB


In [7]:
column_list = df.columns.values.tolist()
for column_name in column_list:
    print(column_name, df[column_name].nunique())

uniqueID 161297
drugName 3436
condition 884
review 112329
rating 10
date 3579
usefulCount 389


In [8]:
df[df.drugName=="Valsartan"]

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1322,206546,Valsartan,High Blood Pressure,"""I took Diovan for two months to reduce blood ...",6,22-Mar-09,40
2939,206572,Valsartan,High Blood Pressure,"""Been on Diovan for 9 years. Started with ring...",6,8-Nov-09,14
4410,206538,Valsartan,High Blood Pressure,"""I was prescribed Valsartan (generic of Diovan...",4,26-Apr-16,46
4928,206462,Valsartan,High Blood Pressure,"""Experience tiredness, no energy.""",8,13-Feb-12,118
...,...,...,...,...,...,...,...
140771,206448,Valsartan,High Blood Pressure,"""This medicine is a lifesaver. My doctor put m...",10,10-Dec-15,37
144062,206577,Valsartan,High Blood Pressure,"""While this medication is working for my high ...",1,15-Aug-09,8
144715,206571,Valsartan,High Blood Pressure,"""My doctor put me on 80mg of Valsartan (Diovan...",7,16-Nov-09,7
157613,206451,Valsartan,High Blood Pressure,"""Taking for 15 years I&#039;m 67. Controls by ...",9,31-Mar-15,49


## Null values in the train set

In [9]:
df.isna().any().any()

True

In [10]:
df.isna().sum().sum()

899

In [11]:
temp = df.isna().sum()/(len(df))*100
print("Column with lowest amount of missings contains {} % missings.".format(temp.min()))
print("Column with highest amount of missings contains {} % missings.".format(temp.max()))


Column with lowest amount of missings contains 0.0 % missings.
Column with highest amount of missings contains 0.5573569254232875 % missings.


In [12]:
df.loc[:, df.isnull().any()].columns

Index(['condition'], dtype='object')

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().values.any()

False

In [15]:
df.isnull().sum().sum()

0

In [16]:
df.shape

(160398, 7)

## Null values in the test set

In [17]:
df_test.isna().any().any()

True

In [18]:
df_test.isna().sum().sum()

295

In [19]:
df_test.dropna(inplace=True)

# Text vectorizer

## Count Vectorizer

In [20]:
y_train=df['condition']
X_train=df['review']
y_test=df_test['condition']
X_test=df_test['review']

In [21]:
count_vectorizer=CountVectorizer(stop_words='english')

In [22]:
count_train=count_vectorizer.fit_transform(X_train.values)
scipy.sparse.save_npz('count_train', count_train)

In [23]:
#Transform the data into a 'document-term matrix'
count_test=count_vectorizer.transform(X_test.values)
type(count_test)

scipy.sparse.csr.csr_matrix

In [24]:
scipy.sparse.save_npz('count_test', count_test)

In [25]:
count_test

<53471x49549 sparse matrix of type '<class 'numpy.int64'>'
	with 1759315 stored elements in Compressed Sparse Row format>

In [26]:
#count_vectorizer.get_feature_names()[:10]

In [27]:
#count_df=pd.DataFrame(count_train.A , columns=count_vectorizer.get_feature_names())
#count_df.head()

In [28]:
# # convert sparse matrix to a dense matrix
# trans_text.toarray()

# # examine the vocabulary and document-term matrix together
# pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

## TfidVectorizer

In [29]:
tfidf_vectorizer= TfidfVectorizer(stop_words="english", max_df=0.7, ngram_range=(1,3), min_df=2, max_features=100000)
#tfidf_vectorizer= TfidfVectorizer(ngram_range=(,), analyzer='char', max_features=)

In [30]:
tfidf_train=tfidf_vectorizer.fit_transform(X_train.values)

In [31]:
scipy.sparse.save_npz('tfidf_train', tfidf_train)

In [32]:
tfidf_test=tfidf_vectorizer.transform(X_test.values)

In [33]:
scipy.sparse.save_npz('tfidf_test', tfidf_test)

In [34]:
len(tfidf_vectorizer.get_feature_names())

100000

In [35]:
#tfidf_train.A[:5]

In [36]:
#tfidf_df=pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
#tfidf_df.head()